In [ ]:
pip install tavily-python


In [ ]:
from tavily import TavilyClient

# Initialize the client
tavily = TavilyClient(api_key="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")

# Perform a search
response = tavily.search(query="What is the current temperature in Denver CO", search_depth="advanced", max_results=3)

#print(response)

# Print each result with title, URL, and content (snippet)
for i, result in enumerate(response.get('results', [])):  # Safe access with .get()
    title = result.get("title", "No title available")
    url = result.get("url", "No URL available")
    content = result.get("content", "No content available")  # Use 'content' instead of 'snippet'

    print(f"{i+1}. {title}\n   URL: {url}\n   Content: {content}\n")

In [ ]:
print(type(response))

In [ ]:
print(response.keys())

In [ ]:
record = response.get('results', [])[0]  # Default to empty list if 'results' key is missing
print(record)
# Print the specific record in a formatted JSON style
#print(json.dumps(record, indent=4))

In [ ]:
print(type(record))

In [ ]:
print(record.keys())

In [ ]:
import os
from langchain.tools import TavilySearchResults

# Set the API key as an environment variable (if not already set)
os.environ["TAVILY_API_KEY"] = "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"

# Initialize TavilySearchResults (API key will be fetched from environment)
tavily = TavilySearchResults()

# Test the search functionality
query = "What is the weather in Denver CO Today?"
# Run the search query
run_results = tavily.run(
    tool_input=query,
    max_results=3,
    search_depth="advanced",
)


# Print the results
print(run_results)

In [ ]:
print(type(run_results))

In [ ]:
print(len(run_results))

In [ ]:
print(run_results[0])

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.tools import WebSearchResults

# Step 1: Define WebSearch Tool (already configured)
tavily_search = WebSearchResults(tavily_search_func=tavily.search)

# Step 2: Define the prompt template for the LLM
llm_prompt = PromptTemplate(
    input_variables=["query", "web_results"],
    template="""
    You are a helpful assistant. Based on the following search results, please answer the user's question:
    
    Question: {query}

    Search Results:
    {web_results}
    
    Provide a concise, relevant answer based on the results.
    """
)

# Step 3: Initialize the LLM (using OpenAI's GPT-3 for example)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Step 4: Create the LLM Chain
llm_chain = LLMChain(prompt=llm_prompt, llm=llm)

# Step 5: Input Query and Call the WebSearch Tool
query = "What is the current temperature in Denver CO"
web_results = tavily_search.search(query=query, search_depth="advanced", max_results=3)

# Step 6: Pass the results into the chain to generate the final answer
llm_input = {
    "query": query,
    "web_results": "\n".join([f"Source: {result['url']}\nContent: {result['content']}" for result in web_results['results'][:3]])
}

# Step 7: Execute the chain to get the final answer
final_answer = llm_chain.run(llm_input)

# Step 8: Display the result
print(final_answer)


In [ ]:
pip install langgraph


In [1]:
pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
